# Tutorial: How to use EVOMIA

Evolutionary Optimization for Material Identification with Abaqus (EVOMIA) can identify material parameters from experimental data using evolutionary optimization algorithm and Abaqus.

This tutorial will guide you through the process of using EVOMIA.

## Environment Requirements


First, ensure that `abaqus>=2022` is installed on your system. If you don't have `abaqus>=2022` installed, you can download the free student edition from the [Dassault Systems website](https://academy.3ds.com/en/software/abaqus-student-edition/) just for learning purposes.

Second, install `EVOMIA` in your environment using the [installation instructions](https://github.com/han-xudong/EVOMIA) in the repository.

## Setup

Necessary imports for running `EVOMIA`.

In [ ]:
import json
import pandas as pd
import os
import optuna
from optuna.visualization import plot_contour, plot_parallel_coordinate, plot_slice, plot_optimization_history
import plotly.io as pio
pio.renderers.default = "vscode"

## Experimental Data

Experimental data is required as input and ground truth for the optimization algorithm. The data should be in the form of a `.json` file with the following format:

```json
{
    "sample": {
        "input": {
            "boundary": 0
        }
        "output": {
            "force": [0.0, 0.0, 0.0],
            "displacement": [0.0, 0.0, 0.0]
        }
    }
}
```

In `.json` file, there are several `sample` objects representing a experimental data sample, which contains `input` and `output` objects. The `input` object contains the simulation inputs, such as boundary conditions, and the `output` object contains the simulation outputs that are used to calculate the error, such as force and displacement. The `.json` file is named as `{obj_name}_{material}_{output}.json`. In this tutorial, we will use `cylinder_linear_integrated_force.json` as an example.

In [ ]:
gt_list = list(json.load(open('../data/ground_truth/cylinder_linear_integrated_force.json', 'r')).values())
print("Sample number: ", len(gt_list))
pd.concat([pd.DataFrame([gt_list[i]["input"].values() for i in range(len(gt_list))], 
                        columns=list(gt_list[0]["input"].keys())), 
           pd.DataFrame([gt_list[i]["output"].values() for i in range(len(gt_list))], 
                        columns=list(gt_list[0]["output"].keys()))], 
          axis=1)

## Templates

Another important preparation is to generate several templates which is necessary during Abaqus simulation. Here are three templates that are required:

- `{obj_name}_{material}.inp`: The input file generated by Abaqus CAE, which contains the simulation settings, such as boundary conditions, material properties, and mesh information. You need to replace the material parameters with replacable parameters, such as `youngs_modulus`, `poisson_ratio`, etc, as well as the boundary conditions.
- `{material}.json`: The material file that contains the material parameters which are to be optimized. Each parameter should be defined by given a range of values, such as `youngs_modulus: [0.05, 05]`.
- `{output}.json`: The output file contains the output parameters that are used to calculate the error, corresponding to the output data.

All templates are stored in the `templates` folder. It's important to ensure that the templates are correctly generated and stored in the correct folder before running the optimization algorithm.

## Running EVOMIA

The optimization algorithm is implemented based on covariance matrix adaptation evolution strategy (CMA-ES) using `optuna` build-in sampler. The `EVOMIA` class provides a simple interface to the algorithm for identifying material parameters. It allows the user to specify several settings:

- `obj_name`: The name of the object.
- `material`: The material model to be used in Abaqus, including linear elastic, Ogden (n=3), etc.
- `output`: The type of output from Abaqus, including force, node displacement, etc.
- `max_trials`: The maximum number of trials for the optimization algorithm.
- `max_time`: The maximum time for the optimization algorithm.
- `err_threshold`: The error threshold to stop the optimization algorithm.
- `batch_size`: The batch size for Abaqus simulations in parallel.

After checking out the above settings, just run the following code to start the optimization process.

```bash
python evomia.py
```

During the process, the parameters and error will be shown in the console, as well as the trial number for the best parameters. All data will be saved in `databoard.sqlite3`, which can be visualized by right-clicking on the file and selecting `Open in Optuna Dashboard`. You can also follow the next section to visualize the data.

### Loading Data

First, we need to set the name of the study that we want to visualize, as well as the path for saving the visualization.

In [ ]:
# Set study name
study_name = 'example'

# Set save path
save_path = '../out/' + study_name + '/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

Then, we can load the study from the SQLite database.

In [ ]:
# Load study
study = optuna.load_study(storage='sqlite:///../databoard.sqlite3',
                          study_name=study_name,
                          sampler=optuna.samplers.CmaEsSampler())

### Best Parameters

The best parameters and values can be obtained as follows:

In [ ]:
# Get best parameters
best_params = study.best_params
print('Best Parameters:', best_params)

# Get best value
best_value = study.best_value
print('Best Value:', best_value)

### Dataframe of Trials

It's available to get and save the dataframe of study trials.

In [ ]:
# Save DataFrame
df = study.trials_dataframe()
df.to_csv(save_path + 'df.csv', index=False)
df

To visualize the optimization process, we can use several built-in functions in `optuna`, and get:

- Optimization history plot.
- Parallel coordinate plot.
- Slice plot.
- Contour plot.

Note that there two ways to visualize the data, including `plotly` and `matplotlib`. `Plotly` provides interactive plots, while `matplotlib` provides static plots. Since saving the `plotly` plots in the notebook is very difficult, it's recommended to use `matplotlib` for saving the plots.

### Optimization History Plot

In [ ]:
# Visualize history with plotly
fig_history = plot_optimization_history(study=study)
fig_history.update_yaxes(type='log')
fig_history.show()

In [ ]:
# Save histoty with matplotlib
import warnings
warnings.filterwarnings("ignore")

ax_history = optuna.visualization.matplotlib.plot_optimization_history(study)
ax_history.set_yscale('log')
ax_history.set_box_aspect(0.5)
ax_history.set_xlabel('Number of trials')
ax_history.set_ylabel('Error')
ax_history.lines[0].set(color='#FF8884', linewidth=1.5, alpha=0.8)
ax_history.collections[0].set(color='#2878B5', sizes=(10,), alpha=1)
ax_history.set_facecolor('white')
ax_history.grid(color='lightgrey')
ax_history.legend(['Error', 'Best Error'])
ax_history.legend().get_frame().set_facecolor('white')
ax_history.spines['top'].set_color('lightgrey')
ax_history.spines['right'].set_color('lightgrey')
ax_history.spines['bottom'].set_color('lightgrey')
ax_history.spines['left'].set_color('lightgrey')
ax_history.figure.set_size_inches(6, 4)
fig_history = ax_history.get_figure()
fig_history.dpi = 300
fig_history.savefig(save_path + 'history.png', dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=True)

### Parallel Coordinate Plot

In [ ]:
# Visualize parallel coordinate with plotly
fig_relation = plot_parallel_coordinate(study=study)
fig_relation.show()

In [ ]:
# Save parallel coordinate with matplotlib
ax_relation = optuna.visualization.matplotlib.plot_parallel_coordinate(study)
ax_relation.collections[0].set(linewidth=0.2, alpha=1)
ax_relation.set_facecolor('white')
ax_relation.figure.set_size_inches(14, 4)
fig_relation = ax_relation.get_figure()
fig_relation.dpi = 300
fig_relation.savefig(save_path + 'relation.png', dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=True)

### Slice Plot

In [ ]:
# Visualize slice with plotly
fig_slice = plot_slice(study=study)
fig_slice.show()

In [ ]:
# Save slice with matplotlib
ax_slice_list = optuna.visualization.matplotlib.plot_slice(study)
for ax_slice in ax_slice_list:
    ax_slice.set_box_aspect(1)
    ax_slice.set_facecolor('white')
    ax_slice.grid(color='lightgrey')
    ax_slice.spines['top'].set_color('lightgrey')
    ax_slice.spines['right'].set_color('lightgrey')
    ax_slice.spines['bottom'].set_color('lightgrey')
    ax_slice.spines['left'].set_color('lightgrey')
    ax_slice.figure.set_size_inches(4*len(ax_slice_list) + 2, 4)
    fig_slice = ax_slice.get_figure()
    fig_slice.dpi = 300
    fig_slice.savefig(save_path + 'slice.png', dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=True)

### Contour Plot

In [ ]:
# Visualize contour with plotly
fig_contour = plot_contour(study=study)
fig_contour.show()

In [ ]:
# Save contour with matplotlib
ax_contour_map = optuna.visualization.matplotlib.plot_contour(study)
if type(ax_contour_map) is not list():
    ax_contour = ax_contour_map
    ax_contour.set_box_aspect(1)
    ax_contour.set_facecolor('white')
    ax_contour.collections[0].set(linewidths=0)
    ax_contour.collections[2].set(color='#ffe664', sizes=(1,), alpha=1)
    ax_contour.figure.set_size_inches(6, 4)
else:
    for i in range(ax_contour_map.shape[0]):
        for j in range(ax_contour_map.shape[1]):
            ax_contour = ax_contour_map[i][j]
            ax_contour.set_box_aspect(1)
            ax_contour.set_facecolor('white')
            if i == j:
                continue
            ax_contour.collections[0].set(linewidths=0)
            ax_contour.collections[2].set(color='#ffe664', sizes=(1,), alpha=1)
            ax_contour.figure.set_size_inches(4*len(ax_slice_list) + 2, 4*len(ax_slice_list))
fig_contour = ax_contour.get_figure()
fig_contour.dpi = 300
fig_contour.savefig(save_path + 'contour.png', dpi=300, bbox_inches='tight', pad_inches=0.1, transparent=True)